In [3]:
# top_logprobs
import sys 
sys.path.append("/home/flowers/work/OpenELM")
from openai import OpenAI
from key import OPENAI_API_KEY
from utils_test_puzzle import get_completion
cfg: dict = {
"temperature": 0.,
# "top_p": 1.,
# TODO: rename config option?
"model": "gpt-3.5-turbo-0125",
"logprobs": True,
"top_logprobs": 5,
}
max_retries=10
timeout=30
client = OpenAI(api_key=OPENAI_API_KEY,max_retries=max_retries, timeout=timeout)


In [4]:
# out=get_completion(client,prompt_fine_finetuning, cfg,temperature=0.0)


In [2]:
n_puzz=30
n_gen=1
int(n_puzz*(n_puzz-1)/2*n_gen)

435

## generate COT prometheus criterion

In [8]:
# few shot examples
crit_1 = """[Does the model guarantee universal suitability and prevent logical inconsistencies in its reasoning methodology for a directive that necessitates a stepwise logical progression? This entails addressing edge cases in programming and mathematical issues, and ensuring there are no counterexamples.]

Score rubrics 1:
Score 1: The rationality of the model's response is entirely nonsensical.
Score 2: The model's response is riddled with significant logical inaccuracies or mistakes.
Score 3: The model's response has a few logical discrepancies or errors, though they are not of major concern.
Score 4: The model's response is logically valid, but some edge cases are overlooked.
Score 5: The model's response is flawless in terms of logic and considers all possible edge cases.
"""
crit_2 = """
[Does the language model adapt its communication style based on the user's needs, such as technical level, language proficiency, and emotional state?]

Score rubrics 2:
Score 1: The language model makes no attempt to adapt its communication style to the user's needs.
Score 2: The language model makes minor adjustments to its communication style, but these are largely ineffective or inappropriate.
Score 3: The language model shows some level of adaptation to the user's needs, but there are still areas of mismatch.
Score 4: The language model adapts its communication style well, with only minor inconsistencies or areas of mismatch.
Score 5: The language model perfectly adapts its communication style to the user's needs, demonstrating a high level of understanding and empathy.
"""
crit_3 = """[Does the central idea or concept of the response stand out as being fresh or innovative?]
Score rubrics 3:
Score 1: The central concept is clichéd and has been explored repeatedly in other works.
Score 2: The central concept has a familiar base but attempts a slightly different angle or twist.
Score 3: The central concept is somewhat original, blending familiar and new elements.
Score 4: The central concept is largely innovative, with just a hint of familiarity.
Score 5: The central concept is groundbreaking, offering a completely fresh perspective.
"""

In [6]:
# criteria for P3
about_new_criteria="""This new criteria and scoring rubrics are designed to evaluate a Python programming puzzle generated by a models.
A Python programming puzzle is defined by two functions, the puzzle f(…) and the solution g(…). f defines an algorithmic challenge, and g solves this challenge. g is a solution to f if and only if f(g()) == True. There can be additional optional keyword arguments to both f and g; these arguments, if they exist, are the same for both functions.

Te puzzle have the following structure:

def f(solution, args=...) -> bool:
    # Python code to test the solution returned by g.
    # This function behaves like a unit test and must return True if the solution is correct, False otherwise.

def g(args=...) -> solution:
    # Python code to generate a solution for the problem.
    # The solution should generalize to all possible args.
    return solution

The evaluation critera for good puzzles are:
Opportunity for learning (how likely is it that a student will learn by trying to solve the puzzle and looking at its solution?);
Difficulty appropriate for the course (is the puzzle too easy?);

For example here are list of puzzles that should get a score of 1:
def f(is_sorted: bool, arr: List[int]) -> bool:
    return is_sorted == all((arr[i] <= arr[i + 1] for i in range(len(arr) - 1)))

def g(arr: List[int]) -> bool:
    return all((arr[i] <= arr[i + 1] for i in range(len(arr) - 1)))
It is a problem that is too easy and does not provide any learning opportunity as the solution is just a copy paste of the problem.

from typing import List
from itertools import permutations

def f(nums: List[int]) -> bool:
    def check_condition(perm):
        # Define your condition here
        return True

    for perm in permutations(nums):
        if check_condition(perm):
            return True
    return False

def g():
    return [1, 2, 3]

assert f(g()) == True
The problem of this puzzle is fundamentally flawed as it always returns True, regardless of the input.


Puzzle that should get a score of 2 or 3:
def f(s: str) -> bool:
    return s == s[::-1]
def g():
    return "racecar" 
assert f(g()) == True
This porblem is easy to understand and it offer low to medium learning opportunity 
"""

In [7]:
new_criteria="""We are brainstorming criteria with which to grade a language model on its responses in
diverse situations.
A Criteria is some useful, real-world objective, and associated rubric for scores 1-5, that
tests a capability.
Here you will see 3 examples of ‘criteria‘, and their scoring rubrics.
Criteria 1:
{crit_1}
Criteria 2:
{crit_2}
Criteria 3:
{crit_3}
Please brainstorm a new criteria and scoring rubrics.
Be creative and create new but useful criteria that people in different settings or industries
might find practical.
Please format the output as same as the above examples.
{about_new_criteria}
New Criteria:
"""

ask_gpt=new_criteria.format(crit_1=crit_1,crit_2=crit_2,crit_3=crit_3,about_new_criteria=about_new_criteria)
print(ask_gpt)

We are brainstorming criteria with which to grade a language model on its responses in
diverse situations.
A Criteria is some useful, real-world objective, and associated rubric for scores 1-5, that
tests a capability.
Here you will see 3 examples of ‘criteria‘, and their scoring rubrics.
Criteria 1:
[Does the model guarantee universal suitability and prevent logical inconsistencies in its reasoning methodology for a directive that necessitates a stepwise logical progression? This entails addressing edge cases in programming and mathematical issues, and ensuring there are no counterexamples.]

Score rubrics 1:
Score 1: The rationality of the model's response is entirely nonsensical.
Score 2: The model's response is riddled with significant logical inaccuracies or mistakes.
Score 3: The model's response has a few logical discrepancies or errors, though they are not of major concern.
Score 4: The model's response is logically valid, but some edge cases are overlooked.
Score 5: The model'

 ### results

In [16]:
prometheus_1="""[Is the Python programming puzzle effectively designed and solvable, with the solution function correctly addressing the algorithmic challenge posed by the puzzle function?]

Score rubrics:
Score 1: The puzzle (f) and solution (g) functions are fundamentally flawed, either due to incorrect logic, making the puzzle unsolvable, or because the solution does not correctly solve the intended challenge.
Score 2: The puzzle is solvable, but the solution is inefficient or partially incorrect, solving the puzzle only under specific conditions or failing to address all potential edge cases.
Score 3: The puzzle is designed well and the solution solves the challenge correctly, but there are minor inefficiencies or lack of optimization in the solution.
Score 4: Both the puzzle and solution are well-designed, with the solution being efficient and correct. However, there might be room for slight improvements in terms of optimization or elegance.
Score 5: The puzzle is ingeniously designed to challenge understanding of specific programming concepts, and the solution is not only correct but also optimized and elegant, showcasing deep algorithmic understanding and creativity.
"""

prometheus_2= """Criteria:
[How effectively does the Python programming puzzle balance complexity and accessibility to foster problem-solving skills without overwhelming the solver? This includes the clarity of the puzzle's description, the creativity and relevance of the problem, and the educational value in terms of programming concepts and logic application.]
Python Programming Puzzles (P3) consists of two functions: a problem function `f` and its corresponding solution `g`. The challenge lies in solving a challenging SAT problem `f` with a function `g` such that `f(g())` evaluates to `True`.

Score rubrics:
Score 1: The puzzle is either overly simplistic or impossibly complex, with unclear instructions that confuse rather than guide the solver. It lacks educational value and fails to engage.
Score 2: The puzzle presents a somewhat clear challenge but falls short in balancing complexity with accessibility. It offers limited educational value and engages the solver to a small extent.
Score 3: The puzzle has a clear set of instructions and is moderately challenging. It introduces relevant programming concepts and logic application, providing a fair educational value and engagement.
Score 4: The puzzle is well-crafted with clear, concise instructions and a cleverly balanced level of complexity. It is engaging and significantly contributes to the solver's understanding of key programming concepts and logic.
Score 5: The puzzle exemplifies an ideal balance of complexity and accessibility, with crystal-clear instructions and a highly relevant, innovative challenge. It maximizes educational value, thoroughly engages the solver, and enhances problem-solving skills in a fun and meaningful way."""

prometheus_2_1 ="""Criteria:
[Is the language model capable of creating Python programming puzzles that are not only innovative but also educational, striking a balance between challenge and learnability?]

This criterion evaluates the model's ability to generate Python programming puzzles (including both the puzzle f(…) and its solution g(…)), assessing them based on their clarity, potential for learning, appropriateness of difficulty, and level of engagement. The goal is to measure the effectiveness of puzzles as educational tools that can stimulate interest and understanding in Python programming.

Score rubrics:

Score 1: The puzzle is confusing, offers no clear learning outcomes, is either too simplistic or excessively complex, and fails to engage or interest the user.

Clarity: The instructions and objectives of the puzzle are unclear or misleading.
Opportunity for learning: The puzzle does not introduce any new concepts or reinforce existing knowledge in a meaningful way.
Difficulty: The puzzle is either too easy, offering no challenge, or too difficult, making it inaccessible for the intended audience.
Engagement: The puzzle is dull or frustrating, with no elements designed to spark interest or curiosity.
Score 2: The puzzle has some elements of clarity and educational value but falls short in difficulty balance and engagement.

Clarity: The puzzle's instructions are somewhat understandable, but key details may be vague or ambiguous.
Opportunity for learning: The puzzle introduces or reinforces programming concepts, but these are not well-integrated into the challenge.
Difficulty: The puzzle's difficulty is inconsistent, with parts that are either too easy or too hard relative to the intended learning outcomes.
Engagement: The puzzle offers minimal elements of fun or intrigue, but these are not enough to sustain interest.
Score 3: The puzzle is adequately clear, provides a fair learning opportunity, is appropriately challenging for its target audience, and has some engaging elements.

Clarity: The puzzle's instructions and goals are clear, with minor areas of ambiguity.
Opportunity for learning: The puzzle offers a good learning opportunity by introducing new concepts or reinforcing existing ones in a context that requires application.
Difficulty: The puzzle is designed with an appropriate level of difficulty, challenging enough to be educational without being discouraging.
Engagement: The puzzle is somewhat engaging, with elements that can capture and hold the user's interest to a reasonable degree.
Score 4: The puzzle is clear, highly educational, has a well-balanced level of difficulty, and is engaging.

Clarity: The puzzle's instructions and objectives are very clear and concise, making it easy to understand what is required.
Opportunity for learning: The puzzle effectively introduces or reinforces important programming concepts in a way that enhances understanding and skills.
Difficulty: The puzzle's difficulty is well-calibrated, providing a challenge that is neither too easy nor too hard, encouraging deep engagement with the material.
Engagement: The puzzle is engaging and enjoyable, with creative elements that stimulate interest and motivation to solve.
Score 5: The puzzle is exemplary in clarity, offers outstanding educational value, perfectly matches the difficulty level for its audience, and is highly engaging and enjoyable.

Clarity: The puzzle's instructions and objectives are perfectly articulated, leaving no room for misinterpretation or confusion.
Opportunity for learning: The puzzle is a superior educational tool, offering deep insights into programming concepts and techniques, and encouraging critical thinking and problem-solving skills.
Difficulty: The puzzle strikes an ideal balance, presenting a challenge that is precisely aligned with the intended learning outcomes and audience's abilities.
Engagement: The puzzle is exceptionally engaging, with innovative and fun elements that greatly enhance the learning experience, making it both memorable and effective.

"""

## prompt 

In [24]:
prompt_instruction_reference="""Your job is to generate a new puzzle that is related to the given score
rubric.
The score rubric:
{CRITERIA}

* Generate a new Python programming puzzle that would be rated highly according to the given score rubric, particularly focusing on educational value, engagement, and the enhancement of problem-solving skills..
Data Generation:"""


In [26]:
print(prompt_instruction_reference.format(CRITERIA=prometheus_2))

Your job is to generate a new puzzle that is related to the given score
rubric.
The score rubric:
Criteria:
[How effectively does the Python programming puzzle balance complexity and accessibility to foster problem-solving skills without overwhelming the solver? This includes the clarity of the puzzle's description, the creativity and relevance of the problem, and the educational value in terms of programming concepts and logic application.]
Python Programming Puzzles (P3) consists of two functions: a problem function `f` and its corresponding solution `g`. The challenge lies in solving a challenging SAT problem `f` with a function `g` such that `f(g())` evaluates to `True`.

Score rubrics:
Score 1: The puzzle is either overly simplistic or impossibly complex, with unclear instructions that confuse rather than guide the solver. It lacks educational value and fails to engage.
Score 2: The puzzle presents a somewhat clear challenge but falls short in balancing complexity with accessibili

In [ ]:
"""Problem:
Given a scenario where a programming instructor wants to create an engaging and educational Python programming puzzle that strikes a perfect balance between complexity and accessibility.
The challenge is to design a puzzle that effectively tests the solver's understanding of intermediate to advance Python concepts without becoming too esoteric or trivial.
The puzzle must also incorporate an element of creativity and relevance to real-world programming scenarios. 
"""
# The ultimate goal is to create a puzzle that would be rated highly according to the given score rubric, particularly focusing on educational value, engagement, and the enhancement of problem-solving skills.
